In [2]:
from google.colab import drive
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import os

In [4]:
def load_relert_data(file_path):
    """Load optimization_data_relert.csv."""
    if not os.path.exists(file_path):
        print(f"Error: File {file_path} does not exist.")
        return None
    try:
        data = pd.read_csv(file_path)
        required_columns = ['Algorithm', 'Function', 'Instance', 'Dimension', 'ERT']
        if not all(col in data.columns for col in required_columns):
            print(f"Error: Missing required columns in {file_path}. Found: {data.columns.tolist()}")
            return None
        data['ERT'] = pd.to_numeric(data['ERT'], errors='coerce')
        return data
    except Exception as e:
        print(f"Error loading data: {e}")
        return None

In [5]:
def select_top_algorithms(data, top_n=10):
    """Calculate mean ERT per algorithm and select top N algorithms."""
    if data is None:
        return None
    try:
        # Replace infinite ERT values with a large number to avoid skewing mean
        data['ERT'] = data['ERT'].replace([float('inf')], 1e6)
        # Calculate mean ERT per algorithm
        mean_ert = data.groupby('Algorithm')['ERT'].mean().reset_index()
        # Sort by mean ERT (ascending) and select top N
        top_algorithms = mean_ert.sort_values('ERT').head(top_n)
        return top_algorithms
    except KeyError as e:
        print(f"Error: Missing column {e}")
        return None

In [6]:
def save_results(top_algorithms, output_path):
    """Save top algorithms to CSV."""
    if top_algorithms is not None:
        top_algorithms.to_csv(output_path, index=False)
        print(f"Top algorithms saved to {output_path}")
    else:
        print("No results to save.")

In [7]:
# Main execution
if __name__ == "__main__":
    relert_file = '/content/drive/MyDrive/Wail-Projet-F/Data-00/optimization_data_relert.csv'
    output_file = '/content/drive/MyDrive/Wail-Projet-F/Data-02/top_10_algorithms.csv'

    data = load_relert_data(relert_file)
    if data is not None:
        top_algorithms = select_top_algorithms(data, top_n=10)
        if top_algorithms is not None:
            print("\nTop 10 Algorithms by Mean ERT:")
            print(top_algorithms)
            save_results(top_algorithms, output_file)
        else:
            print("Failed to select top algorithms.")
    else:
        print("Failed to load data.")


Top 10 Algorithms by Mean ERT:
                   Algorithm            ERT
6      CMAES-APOP-KMA_Nguyen  125228.426712
16  DE-BFGS_voglis_noiseless  131625.600390
21         ad-CMA-ES_Gissler  131977.616029
22        adm-CMA-ES_Gissler  133066.749552
27          s-CMA-ES_Gissler  134764.343640
26         dm-CMA-ES_Gissler  136283.783888
19                  a-CMA-ES  138845.480604
20          a-CMA-ES_Gissler  138845.480604
30         sd-CMA-ES_Gissler  140962.768889
0               BIPOP-CMA-ES  142401.886531
Top algorithms saved to /content/drive/MyDrive/Wail-Projet-F/Data-02/top_10_algorithms.csv
